In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division
import scipy
import tensorflow as tf
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose, Deconv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
from data_loader import DataLoader
import numpy as np
import os

Using TensorFlow backend.


In [3]:
img_rows = 32
img_cols = 32
channels = 3
img_shape = (img_rows, img_cols, channels)
gen_inshape = (img_rows, img_cols, 1)
gen_outshape = (img_rows, img_cols, 3)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)


dataset_name = 'cifar-10-batches-img'
data_loader = DataLoader(dataset_name=dataset_name,
                         img_res=(img_rows, img_cols))

In [4]:
imgs_A= data_loader.load_data(batch_size=10, is_testing=False)

In [6]:
def build_generator():
    inputs = Input(shape=gen_inshape)

    leaky_relu = tf.nn.leaky_relu
    e1 = Conv2D(64,kernel_size = (4,4),strides = 1, activation = leaky_relu,padding='same')(inputs)
    e1 = BatchNormalization()(e1)
    e2 = Conv2D(128,kernel_size = (4,4),strides = 2, activation = leaky_relu,padding='same')(e1)
    e2 = BatchNormalization()(e2)
    e3 = Conv2D(256,kernel_size = (4,4),strides = 2, activation = leaky_relu,padding='same')(e2)
    e3 = BatchNormalization()(e3)
    e4 = Conv2D(512,kernel_size = (4,4),strides = 2, activation = leaky_relu,padding='same')(e3)
    e4 = BatchNormalization()(e4)
    e5 = Conv2D(512,kernel_size = (4,4),strides = 2, activation = leaky_relu,padding='same')(e4)
    e5 = BatchNormalization()(e5)
    
    d1 = Deconv2D(512,kernel_size = (4,4),strides = 2, activation = 'relu',padding='same')(e5)
    d1 = BatchNormalization()(d1)
    d1 = Dropout(0.5)(d1)
    d1 = concatenate([d1,e4],axis = 3)
    d2 = Deconv2D(256,kernel_size = (4,4),strides = 2, activation = 'relu',padding='same')(d1)
    d2 = BatchNormalization()(d2)
    d2 = Dropout(0.5)(d2)
    d2 = concatenate([d2,e3],axis = 3)
    d3 = Deconv2D(128,kernel_size = (4,4),strides = 2, activation = 'relu',padding='same')(d2)
    d3 = BatchNormalization()(d3)
    d3 = concatenate([d3,e2],axis = 3)
    d4 = Deconv2D(64,kernel_size = (4,4),strides = 2, activation = 'relu',padding='same')(d3)
    d4 = BatchNormalization()(d4)
    d4 = concatenate([d4,e1],axis = 3)
    
    d5 = Conv2D(3,kernel_size = (1,1),strides = 1,activation='tanh')(d4)


    return Model(inputs, d5)

In [7]:
def build_discriminator():
    img_A = Input(shape=img_shape)
    img_B = Input(shape=gen_inshape)
    leaky_relu = tf.nn.leaky_relu
    combined_imgs = Concatenate(axis=3)([img_A, img_B])

    d1 = Conv2D(64, kernel_size=(4,4), strides = 2, activation = leaky_relu, padding="same")(combined_imgs)
    d2 = Conv2D(128, kernel_size=(4,4), strides = 2, activation = leaky_relu, padding="same")(d1)
    d2 = BatchNormalization()(d2)
    d3 = Conv2D(256, kernel_size=(4,4), strides = 2, activation = leaky_relu, padding="same")(d2)
    d3 = BatchNormalization()(d3)
    d4 = Conv2D(512, kernel_size=(4,4), strides = 1, activation = leaky_relu, padding="same")(d3)
    d4 = BatchNormalization()(d4)
    d5 = Conv2D(1, kernel_size=(4,4), strides = 1, activation = 'sigmoid', padding="same")(d4)

    return Model([img_A, img_B], d5)

In [8]:
generator = build_generator()
discriminator = build_discriminator()

In [9]:
from skimage import io, color
def keras_preprocess(img):
    lab = color.rgb2lab(img/255.0)
    L,A,B=cv2.split(lab)
    L = L / 50 - 1
    A = A / 110
    B = B / 110
    ret = cv2.merge((L,A,B))
    return ret

In [10]:
from skimage import io, color
def keras_postprocess(img):
    L,A,B=cv2.split(img)
    L = (L + 1) / 2 * 100
    A = A * 110
    B = B * 110
    lab = cv2.merge((L,A,B))
    ret = color.lab2rgb(lab)
    ret = ret*255
    return ret

In [11]:
from keras import losses
gen_optimizer = Adam(0.005, 0.5)
dis_optimizer = Adam(0.0005, 0.5)

discriminator.compile(loss='binary_crossentropy',
            optimizer=dis_optimizer)

img_A = Input(shape=img_shape)
img_B = Input(shape=gen_inshape)

fake_A = generator(img_B)

discriminator.trainable = False

valid = discriminator([fake_A, img_B])



combined = Model(inputs=[img_A, img_B], outputs=[valid, fake_A])
combined.compile(loss=['binary_crossentropy', 'mae'],
                      loss_weights=[1, 100],
                      optimizer=gen_optimizer)

In [14]:
import cv2
def sample_images(epoch, batch_i):
    r, c = 3, 3

    img = data_loader.load_data(batch_size=3, is_testing=True)
    
    img = img.astype('float64')
    imgs_B = []
    for i in img:
        imgs_B.append(color.rgb2grey(i).reshape(img_rows,img_cols,1))
    imgs_B = np.array(imgs_B)
    
    fake_A = generator.predict(imgs_B)

    titles = ['Gray', 'Generated', 'Original']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        axs[i,0].imshow(imgs_B[i].reshape(img_rows,img_cols),cmap='gray')
        axs[i,0].set_title(titles[0])
        axs[i,0].axis('off')
        
        axs[i,1].imshow(keras_postprocess(fake_A[i])/255.0)
        axs[i,1].set_title(titles[1])
        axs[i,1].axis('off')
        
        axs[i,2].imshow(img[i].astype('int'))
        axs[i,2].set_title(titles[2])
        axs[i,2].axis('off')
    fig.savefig("images_samples/%d_%d.png" % (epoch, batch_i))
    plt.close()

In [17]:
import cv2
from tqdm import tqdm
def train(epochs, batch_size=1, sample_interval=50):

    start_time = datetime.datetime.now()
    fake = np.zeros((batch_size,) + (4,4,1))
    valid = np.ones((batch_size,) + (4,4,1))
    for epoch in range(epochs):
        print (epoch)
        for batch_i, img in enumerate(tqdm(data_loader.load_batch(batch_size))):
            img = img.astype('float64')
            imgs_A = []
            imgs_B = []
            for i in img:
                imgs_A.append(keras_preprocess(i))
                imgs_B.append(color.rgb2grey(i).reshape(img_rows,img_cols,1))
            imgs_A = np.array(imgs_A)
            imgs_B = np.array(imgs_B)
            fake_A = generator.predict(imgs_B)
            d_loss_real = discriminator.train_on_batch([imgs_A, imgs_B], valid)
            d_loss_fake = discriminator.train_on_batch([fake_A, imgs_B], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, imgs_A])

            elapsed_time = datetime.datetime.now() - start_time

            #print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f] time: %s" % (epoch, epochs,
            #                                                        batch_i, data_loader.n_batches,
            #                                                        d_loss,
            #                                                        g_loss[0],
            #                                                        elapsed_time))

            if batch_i % sample_interval == 0:
                sample_images(epoch, batch_i)



In [18]:
train(epochs=200, batch_size=256, sample_interval=200)

0it [00:00, ?it/s]

0


/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 267 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 260 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
/usr/local/lib/python2.7/dist-packages/skimage/color/colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 253 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
194it [01:30,  2.30it/s]
0it [00:00, ?it/s]

1


194it [01:23,  2.30it/s]
0it [00:00, ?it/s]

...

143


194it [01:24,  2.38it/s]
0it [00:00, ?it/s]

144


53it [00:23,  2.36it/s]

KeyboardInterrupt: 